# Исследование по рекомендации тарифного плана

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

## 1. Откройте и изучите файл

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('c:/Users/User/Downloads/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df['is_ultra'].value_counts()

is_ultra
0    2229
1     985
Name: count, dtype: int64

#### Вывод: 
- Данные успешно загружены из файлов. Данные соответствуют описанию.
- В данных отсутствуют пропуски, наименование столбцов корректно.
- Около 70% пользователей используют тариф «Смарт», а 30% — тариф «Ультра».

## 2. Разбейте данные на выборки

Так как спрятанной тестовой выборки у нас нету, разделим исходные данные на три выборки: обучающую, валидационную и тестовую.

In [4]:
df_train, df_valid = train_test_split(df, train_size=0.6, random_state=12345)
df_valid, df_test = train_test_split(df_valid, train_size=0.5, random_state=12345)

In [5]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

#### Вывод: 
- На этом этапе проекта разбили исходные данные на обучающую, валидационную и тестовую выборки, и в каждой из них выделили признаки и целевой признак.

## 3. Исследуйте модели

Так как перед нами стоит задача бинарной классификации — проанализировать поведение клиентов и предложить пользователям новый тариф — на этом этапе рассмотрим следующие модели классификации:

- Дерево принятия решений (Decision Tree Classifier)
- Случайный лес (Random Forest Classifier)
- Логистическая регрессия (Logistic Regression)

### Дерево принятия решений (Decision Tree Classifier)

Сначала проверим, какой результат дает данная модель без смены гиперпараметров, а потом, изменяя гиперпараметры, попробуем улучшить работу модели.

In [6]:
model_dtc = DecisionTreeClassifier(random_state=12345)
model_dtc.fit(features_train, target_train)
predictions_valid = model_dtc.predict(features_valid)
accuracy_dtc = accuracy_score(target_valid, predictions_valid)

print('Result:', accuracy_dtc)

Result: 0.713841368584759


In [7]:
results_from_box = []
results_from_box.append({'DecisionTreeClassifier': accuracy_dtc})
results_from_box

[{'DecisionTreeClassifier': 0.713841368584759}]

Настроим гиперпараметр max-depth (глубина деревьев) и посмотрим, как это улучшит работу модели. Для этого в цикле переберем значения глубины деревьев в диапазоне от 1 до 5.

In [8]:
results_dtc = []

for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)

    results_dtc.append({'Model': 'DecisionTreeClassifier', 
                    'Hyperparameters': {'random_state': 12345, 'max_depth':depth}, 
                    'Accuracy score': accuracy_valid})

In [9]:
df_dtc = pd.DataFrame.from_dict(results_dtc)
df_dtc

,Model,Hyperparameters,Accuracy score
0,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 1}",0.754277
1,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 2}",0.782271
2,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 3}",0.785381
3,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 4}",0.779160
4,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 5}",0.779160


Лучший результат сохраним в переменной best_results.

In [10]:
best_results = []
best_results.append(df_dtc.loc[3])

#### Вывод:
- Видим, что наилучший результат модель дает при глубине 3 (accuracy_score = ~0,78). Такое значение гиперпараметра max_depth позволит модели обучится, но в тоже время не позволит ей разрастись, что могло бы привести модель к переобучению.

### Случайный лес (Random Forest Classifier)

Проверим, как работает модель «из коробки», а результат сохраним в переменной results_from_box.

In [11]:
model_rfc = RandomForestClassifier(random_state=12345) 
model_rfc.fit(features_train, target_train)  

predictions_rfc = model_rfc.predict(features_valid) 
accuracy_rfc = accuracy_score(target_valid, predictions_rfc)

print('Result:', accuracy_rfc)

Result: 0.7853810264385692


In [12]:
results_from_box.append({'RandomForestClassifier': accuracy_rfc})
results_from_box

[{'DecisionTreeClassifier': 0.713841368584759},
 {'RandomForestClassifier': 0.7853810264385692}]

Видим, что результат значительно лучше, чем показала модель DecisionTreeClassifier.

В цикле настроим гиперпараметры n_estimators и max_depth. Гиперпараметр n_estimators показывает количество деревьев в "лесу" (чем выше показатель, тем больше модель разрастается и медленнее обучается), а max_depth — глубину деревьев.

In [13]:
results_rfc = []

for depth in range(1,11):
    
    for estimator in range(10, 101, 10):
        
        model = RandomForestClassifier(random_state=12345, n_estimators=estimator, max_depth=depth) 
        model.fit(features_train, target_train)
    
        predictions_valid = model.predict(features_valid)
        accuracy_valid = accuracy_score(target_valid, predictions_valid)
       
        results_rfc.append({'Model': 'RandomForestClassifier', 
                            'Hyperparameters': {'random_state': 12345, 
                                                'n_estimators': estimator, 
                                                'max_depth':depth}, 
                            'Accuracy score': accuracy_valid})

Все результаты сохранены в переменной results_rfc. Выведем на экран ту комбинацию гиперпараметров, которая дает самый высокий показатель аccuracy_score.

In [14]:
df_rfc = pd.DataFrame.from_dict(results_rfc)
df_rfc[df_rfc['Accuracy score']==df_rfc['Accuracy score'].max()]

,Model,Hyperparameters,Accuracy score
73,RandomForestClassifier,"{'random_state': 12345, 'n_estimators': 40, 'm...",0.808709


In [15]:
best_results.append(df_rfc.loc[73])
df_rfc['Hyperparameters'].loc[73]

{'random_state': 12345, 'n_estimators': 40, 'max_depth': 8}

#### Вывод:
- Наилучший результат модель показывает, когда гиперпараметр n_estimators равен 40, а max_depth = 8 (accuracy_score = ~0.80). Настройка гиперпараметров позволила улучшить работу модели.

### Логистическая регрессия

Проверим, как работает модель «из коробки», а результат сохраним в переменной results_from_box.

In [16]:
model_lr_frombox = LogisticRegression() 
model_lr_frombox.fit(features_train, target_train)
predictions_lr = model_lr_frombox.predict(features_valid)

accuracy_lr = accuracy_score(target_valid, predictions_lr)
accuracy_lr

0.7107309486780715

In [17]:
results_from_box.append({'LogisticRegression': accuracy_lr})

Поменяем гиперпараметры.

In [18]:
results_lr = []

model_lr = LogisticRegression(random_state=12345, solver='lbfgs', penalty='l2') 
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)

accuracy_lr = accuracy_score(target_valid, predictions_lr)

results_lr.append({'Model': 'LogisticRegression', 
                   'Hyperparameters': {'random_state': 12345, 'solver':'lbfgs', 'penalty': 'l2'}, 
                   'Accuracy score': accuracy_lr})

accuracy_lr

0.7107309486780715

In [19]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1')  
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)

accuracy_lr = accuracy_score(target_valid, predictions_lr)

results_lr.append({'Model': 'LogisticRegression', 
                   'Hyperparameters': {'random_state': 12345, 'solver':'liblinear', 'penalty': 'l1'}, 
                   'Accuracy score': accuracy_lr})

accuracy_lr

0.7573872472783826

#### Вывод:
- При смене значений гиперпараметра penalty на l1, а гиперпараметра solver на liblinear показатель accuracy_score стал лучше. Добавим этот результат в переменную best_results и перейдем к сравнению моделей.

In [20]:
df_lr = pd.DataFrame.from_dict(results_lr)
best_results.append(df_lr.loc[1])

### Сравнение моделей

Сравним результаты разных моделей. Сначала выведем на экран, какие результаты дает каждая из моделей без настройки гиперпараметров.

In [21]:
pd.DataFrame(results_from_box)

,DecisionTreeClassifier,RandomForestClassifier,LogisticRegression
0,0.713841,NaN,NaN
1,NaN,0.785381,NaN
2,NaN,NaN,0.710731


Видим, что самый лучший результат accuracy_score показывает модель случайного леса (0.78), а самый худший — модель логистической регрессии (0.71).

Выведем на экран ранее отобранные модели, которые показали самый лучший результат в каждой из модели. Из них выберем лучшую и протестируем ее на тестовой выборке.

In [25]:
pd.DataFrame(best_results)

,Model,Hyperparameters,Accuracy score
3,DecisionTreeClassifier,"{'random_state': 12345, 'max_depth': 4}",0.779160
73,RandomForestClassifier,"{'random_state': 12345, 'n_estimators': 40, 'm...",0.808709
1,LogisticRegression,"{'random_state': 12345, 'solver': 'liblinear',...",0.757387


#### Вывод:

- Наилучший результат показала модель случайного леса с гиперпараметрами n_estimators = 40 и max_depth = 8. Именно эту модель позже протестируем на тестовой выборке.

- Результат: на данном этапе проекта мы исследовали три модели классификации — дерево принятия решений, случайный лес и логистическая регрессия.

- Каждую из моделей обучили на обучающей выборке и проверили на валидационной выборке. Наилучший результат показала модель случайного леса (показатель accuracy_score без настройки гиперпараметров = 0.78, с настройкой гиперпараметров = 0.80).

- Далее проверим модель случайного леса с гиперпараметрами n_estimators = 40 и max_depth = 8 на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

In [26]:
model_final = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=8)
model_final.fit(features_train, target_train)  
predictions_final = model_final.predict(features_test)

accuracy_final = accuracy_score(target_test, predictions_final)
accuracy_final 

0.7962674961119751

#### Вывод:

- Проверка модели на тестовой выборке показала, что модель работает достаточно хорошо — она способна правильно предсказать почти 80% ответов.

## 5. Проверьте модели на адекватность

Для проверки модели на адекватность будем использовать классификатор DummyClassifier из библиотеки Skelarn.

Для проверки модели отберем 3 стратегии — stratified, most_frequent и uniform. Если их способность предсказать результат будет лучше чем у настроенной модели случайного леса, будем считать, что наша модель работает плохо и ее необходимо настраивать заного.

In [27]:
strategies = ['stratified', 'most_frequent', 'uniform'] 
  
dummy_results = [] 
for strategy in strategies: 
    dc = DummyClassifier(strategy = strategy, random_state=12345)
    
    dc.fit(features_train, target_train) 
    result = dc.score(features_test, target_test) 
    dummy_results.append({strategy: result}) 

pd.DataFrame(dummy_results) 

,stratified,most_frequent,uniform
0,0.536547,NaN,NaN
1,NaN,0.684292,NaN
2,NaN,NaN,0.482115


У финальной модели случайного леса показатель accuracy_score был 0.79, что гораздо выше, чем те показатели, которые выдал классификатор DummyClassifier(). Соответсвенно, можем сделать вывод, что модель случайного леса работает достаточно хорошо и ее можно использовать на практике.


#### Вывод:

- Модель случайного леса с гиперпараметрами n_estimators = 40 и max_depth = 8 работает лучше, чем стратегии случайного прогнозирования классификатора DummyClassifier().

## Общий вывод
Проверка модели на тестовой выборке показала, что модель работает достаточно хорошо — она способна правильно предсказать около 80% ответов. Можем считать, что цель проекта достигнута.